In [4]:
import plotly.graph_objs as go

def update_graphs():
    global buffer_data
    
    # Convert buffer_data to separate lists for X, Y, Z
    x_data = [data[0] for data in buffer_data]
    y_data = [data[1] for data in buffer_data]
    z_data = [data[2] for data in buffer_data]
    
    # Plot the data
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list(range(len(x_data))), y=x_data, mode='lines', name='X Axis'))
    fig.add_trace(go.Scatter(x=list(range(len(y_data))), y=y_data, mode='lines', name='Y Axis'))
    fig.add_trace(go.Scatter(x=list(range(len(z_data))), y=z_data, mode='lines', name='Z Axis'))
    
    # Save the plot
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    fig.write_image(f'plot_{timestamp}.png')
    
    # Save the data to CSV
    with open(f'data_{timestamp}.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['X', 'Y', 'Z'])
        writer.writerows(buffer_data)
    
    # Update the Plotly Dash app
    # Assuming `app` is your Dash app and `graph_id` is the ID of the graph component
    app.update_layout(
        go.Figure(data=[
            go.Scatter(x=list(range(len(x_data))), y=x_data, mode='lines', name='X Axis'),
            go.Scatter(x=list(range(len(y_data))), y=y_data, mode='lines', name='Y Axis'),
            go.Scatter(x=list(range(len(z_data))), y=z_data, mode='lines', name='Z Axis')
        ])
    )


In [6]:
from collections import deque
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import numpy as np
from datetime import datetime
import csv

def create_live_update_dash(data_fetcher, graph_id='live-graph', refresh_rate=1000, max_samples=200, port=8050):
    """
    Creates a live updating Dash application for continuous data monitoring.

    :param data_fetcher: A callable that fetches new data points (returns a tuple/list of [x, y, z]).
    :param graph_id: The ID for the Dash graph element.
    :param refresh_rate: Time interval (in milliseconds) for graph updates.
    :param max_samples: Maximum number of data samples to display on the graph.
    :param port: Port number for running the Dash server.
    """
    # Buffer to hold incoming data points
    data_buffer = deque(maxlen=max_samples)

    # Initialize Dash app
    app = dash.Dash(__name__)
    app.layout = html.Div([
        dcc.Graph(id=graph_id),
        dcc.Interval(id='interval-component', interval=refresh_rate, n_intervals=0)
    ])

    @app.callback(
        Output(graph_id, 'figure'),
        [Input('interval-component', 'n_intervals')]
    )
    def update_graph(n):
        # Fetch new data from the data source
        new_data = data_fetcher()

        if new_data:
            data_buffer.append(new_data)  # Add new data to the buffer

        # Extract X, Y, Z components from the buffer
        x_data = [point[0] for point in data_buffer]
        y_data = [point[1] for point in data_buffer]
        z_data = [point[2] for point in data_buffer]

        # Create the figure for the graph
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=list(range(len(x_data))), y=x_data, mode='lines+markers', name='X Axis'))
        fig.add_trace(go.Scatter(x=list(range(len(y_data))), y=y_data, mode='lines+markers', name='Y Axis'))
        fig.add_trace(go.Scatter(x=list(range(len(z_data))), y=z_data, mode='lines+markers', name='Z Axis'))

        # Set graph layout
        fig.update_layout(
            title='Live Accelerometer Data',
            xaxis=dict(title='Sample Index'),
            yaxis=dict(title='Values'),
            uirevision='constant'  # Prevent auto-scaling during updates
        )

        return fig

    # Run the Dash server
    app.run_server(debug=True, port=port)

# Example data fetcher function to simulate incoming data
def mock_data_fetcher():
    # Simulate continuous data points; replace with real data fetching logic
    return [np.random.randn(), np.random.randn(), np.random.randn()]

# Example of how to use the reusable function
create_live_update_dash(data_fetcher=mock_data_fetcher, refresh_rate=500, max_samples=150, port=8060)



In [3]:
import pandas as pd

# Load CSV data
data = pd.read_csv('Python_Accelerometer_Combined.csv')

# Statistical measures
mean_x = data['X_value'].mean()
mean_y = data['Y_value'].mean()
mean_z = data['Z_value'].mean()
variance_x = data['X_value'].var()
variance_y = data['Y_value'].var()
variance_z = data['Z_value'].var()

print(f"Mean X: {mean_x}, Variance X: {variance_x}")
print(f"Mean Y: {mean_y}, Variance Y: {variance_y}")
print(f"Mean Z: {mean_z}, Variance Z: {variance_z}")


Mean X: -0.0003091831399936869, Variance X: 0.001566523731599973
Mean Y: -0.032519716205018936, Variance Y: 3.091310641636765e-05
Mean Z: -0.9956202651515151, Variance Z: 3.987440756472044e-06
